In [1]:
# !pip install gym
# !pip install pygame

In [2]:
import gym
from collections import deque
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam
import numpy as np
import random


In [3]:
# env = gym.make('CartPole-v1', render_mode="human")

# print(env.action_space)
# print(env.action_space.sample())
# print(env.reward_range)
# print(env.observation_space)

# env.reset()
# for t in range(1000):
#     sample = env.action_space.sample()
#     result = env.step(sample)
#     env.render()

# env.close()

In [4]:
# env = gym.make('CartPole-v1', render_mode="human")

# for e in range(20): # episodes
#     observation = env.reset()

#     for t in range(50):
#         env.render()
#         action = env.action_space.sample()
#         observation, reward, done, info, prob = env.step(action)
#         # print(observation, reward, done, info, prob)
#         if done:
#             print(f'Game episode: {e}/{20}, High Score: {t}')
#             break

# env.close()
# print('All games finished!!')

In [5]:
class Agent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95 # Discount factor
        # Exploration vs Exploitation trait off.
        # Exploration: Good in the beginning, to try out various actions 
        # Exploitation: Good in the end, to apply best actions from the past experiences(memory) 
        self.epsilon=1.0 # 100% exploration in the beginning
        self.epsilon_decay=0.995
        self.epsilon_min=0.01 
        self.learning_rate=0.001
        self.model = self._create_model()

    def _create_model(self):
        model = Sequential()
        model.add(Dense(24, activation='relu', input_dim=self.state_size))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001))
        return model

    def remember(self, state, action, reward, next_state, done):
        # remember from past
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        # epsilon greedy method
        if np.random.rand() <= self.epsilon:
            # take a random action
            return np.random.randint(1, self.action_size)
        else:
            # take a suitable action
            return np.argmax(self.model.predict(state)[0])

    def train(self, batch_size=32):
        # training using a replay buffer
        minibatch = random.sample(self.memory, batch_size)
        for experience in minibatch:
            s, a, r, s_, d = experience

            # X, Y = s(state), r_(expected_reward)
            if not d:
                # game is not yet over, use bellman equation to approx the target value of reward
                target = r + self.gamma * np.amax(self.model.predict(s_)[0])
            else:
                target = r
            target_f = self.model.predict(s)
            target_f[0][action] = target 

            self.model.fit(s, target_f, epochs=1, verbose=0)

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)               


In [6]:
n_episodes = 1000
output_dir = 'cartpole_model/'

agent = Agent(state_size=4, action_size=2)
done = False

In [7]:
env = gym.make('CartPole-v1', render_mode="human")


for e in range(n_episodes):
    state = env.reset()[0]
    state = np.reshape(state, (1, agent.state_size))

    for t in range(200):
        env.render()
        action = agent.act(state)
        next_state, reward, done, info, prob = env.step(action)
        reward = reward if not done else -10
        next_state = np.reshape(next_state, [1, agent.state_size])
        agent.remember(state, action, reward, next_state, done)

        if done:
            print(f'Game episode :{e}/{n_episodes}, High Score: {t} Exploration Rate: {agent.epsilon}')
            break 
    if len(agent.memory) > 32:
        agent.train()
    if e%50 == 0:
        agent.save(name=output_dir + "weigths_" + f'{e}'+".hdf5")

print("Deep Q-Learner model trained")
env.close()

C:\Users\hhars\AppData\Local\Programs\Python\Python311\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Game episode :0/1000, High Score: 9 Exploration Rate: 1.0
Game episode :1/1000, High Score: 8 Exploration Rate: 1.0
Game episode :2/1000, High Score: 9 Exploration Rate: 1.0
Game episode :3/1000, High Score: 9 Exploration Rate: 1.0
1/1 [==============================] - 0s 22ms/step

1/1 [==============================] - 0s 24ms/step
Game episode :4/1000, High Score: 8 Exploration Rate: 0.995
1/1 [==============================] - 0s 31ms/step
Game episode :5/1000, High Score: 9 Exploration Rate: 0.990025
1/1 [==============================] - 0s 26ms/step
Game episode :6/1000, High Score: 7 Exploration Rate: 0.985074875
1/1 [==============================] - 0s 24ms/step
Game episode :7/1000, High Score: 9 Exploration Rate: 0.9801495006250001
1/1 [==============================] - 0s 23ms/step
Game episode :8/1000, High Score: 10 Exploration Rate: 0.9752487531218751
1/1 [==============================] - 0s 26ms/step
Game episode :9/1000, High Score: 9 Exploration Rate: 0.97037250935

KeyboardInterrupt: 